# 部署与服务你的 LLM 应用

---

现在你已经对 LLM 进行了微调并完成评估，是时候把它投入生产环境了。本节将带你一步步把模型部署成可访问、可扩展的服务，让它真正为用户创造价值。

在整个课程中，我们一直通过 [**Model Studio**](https://modelstudio.console.alibabacloud.com/)（如 `qwen-plus`）和 [**DashScope**](https://github.com/dashscope)（如 `text-embedding-v2`、`text-embedding-v3`）提供的 API 使用 LLM。这是一种高效的捷径，能节省在云端部署和排障模型的大量时间。

不过，在某些场景下你可能希望部署自己的模型。部署意味着把模型从开发阶段迁移到生产环境中，让它可以处理实时推理请求。但部署并不是时时刻刻都必须做的。

让我们一起来探索不同的选项。

```mermaid
graph TD
    subgraph 部署选项
        A[在哪里部署我的 LLM？] --> B{部署<br>路径}

        B --> C[直接调用 API]
        C --> D[Model Studio]
        D --> E[托管服务<br><br>无需部署<br>有速率限制<br>非常适合 Tongyi 模型家族]

        B --> F[自托管]
        F --> G[本地 vLLM]
        G --> H[仅供测试<br><br>单节点<br>无法扩展]

        F --> I[云端部署]
        I --> L[PAI-EAS]
        I --> M[ECS / EGS /<br>ACK / ACS]

        L --> P[弹性推理<br><br>实时且支持高并发]
        M --> Q[完全掌控<br><br>适合复杂或定制化场景]

    end
    
    style C fill:#D6EAF8,stroke:#3498DB
    style F fill:#D5F5E3,stroke:#2ECC71
    style D fill:#EBDEF0,stroke:#8E44AD
    style G fill:#FEF9E7,stroke:#F4D03F
    style L fill:#ABEBC6,stroke:#27AE60
    style M fill:#ABEBC6,stroke:#27AE60
```

## 到目前为止的旅程

从 **TaskFriend** 的萌芽阶段一路走来，你已经取得了长足的进步。

这个最初只会围绕任务进行聊天的简单 AI 助手，如今成长为一个**智能、具备代理能力的系统**，能够查询私有数据，理解诸如“我卡住了——接下来应该做什么？”这样的复杂问题，甚至还能通过查看日历和天气来规划周末行程。

你已经完成了提示词工程，通过重排序和 HyDE 优化了检索，使用 Ragas 评估了性能，并构建了能代表用户行动的多智能体工作流。

现在，是时候把 **TaskFriend** 从开发环境带入现实世界了。

毕竟，再聪明的 AI 如果没有部署上线，也无法帮助任何人。

当然，部署不只是“把它放到线上”这么简单，而是需要做出明智的取舍：  
* 你是应该为了简化流程选择托管 API，还是为了掌控力选择自托管？  
* 如何在规模化场景下高效地提供模型服务？  
* 运行一个 LLM 应用在生产环境中意味着什么？

本章将闭环整个流程——从创意、原型到**可部署、贴近生产的 AI 应用**。

## 学习目标

* 了解在什么情况下应该**部署**你的 LLM 应用，以及哪些情况下可以**无需部署**。
* 比较 **托管服务**（例如 Model Studio）与 **自托管方案**（例如 vLLM、PAI-EAS）的特点。
* 学习如何借助 **vLLM** 等高性能推理引擎高效提供模型服务。
* 通过 OpenAI 兼容的 API 运行一次**本地部署**，加快测试速度。
* 使用 **PAI-EAS** 在云端部署，实现弹性、支持高并发的服务能力。
* 在真实环境中评估 **延迟、吞吐量和可靠性**。
* 在生产环境中就 **成本、可扩展性与掌控力** 做出明智决策。

# 轻量化的“部署”

---

## 完全跳过部署：直接调用 API

先从一个大胆的想法开始：你并不总是需要部署模型。如果你使用的是 [Model Studio](https://modelstudio.console.alibabacloud.com/) 上的预训练云托管模型，可以完全跳过部署环节。

在整个 **[LMP-C01] LLM Engineer (Professional)** 课程中，我们一直都是这么做的。你并没有部署使用过的模型——它们早已由阿里云的 Model Studio 托管、扩展并维护。

这就是托管 LLM API 的威力所在。

你只需发送请求，就能收到响应。无需服务器，也不用为扩缩容头疼。

### 为什么这种方式很好用

> 想象一下，用 Gmail 收发邮件，而不是自己搭建邮件服务器。

| 优势 | 对你意味着什么 |
|--------|------------------------|
| **零部署工作量** | 无需运维、容器或 GPU 环境配置 |
| **按量计费** | 只为消耗的 Token 付费，没有闲置 GPU 成本 |
| **自动扩缩容** | 流量高峰时无需额外配置即可应对 |
| **可靠性内置** | 自带高可用、服务等级协议与监控 |

**以下场景再合适不过：**  
* 原型和概念验证
* 低到中等流量的应用
* 想把精力放在产品而非基础设施的团队

> **⚠️ 限制：**  
> API 可能会受到速率限制（例如 QPM：每分钟请求数，TPM：每分钟 Token 数）。超出配额会导致请求失败。

如果你的模型是**自定义、微调后的，或不在 Model Studio 覆盖范围内**，那就需要自己动手了。

# 在阿里云上部署你的 LLM 应用

---

随着技术的发展，越来越多的组织和个人不再采购大量高性能服务器，而是把云服务作为部署 LLM 的主要途径。这种方式在多个维度上都有显著优势：

*   **硬件成本高**：自行购置大量高性能服务器需要巨额前期投入。
*   **运维成本高**：日常运维（监控、升级、排障）需要极高的专业水平。
*   **可靠性难保障**：服务稳定性高度依赖运维人员能力和项目预算，有限成本下很难做到高可用。
*   **灵活性不足**：受限于固定硬件资源，无法根据实际需求灵活调整，容易出现性能不足或资源浪费。

阿里云提供了丰富的服务，如 **Model Studio**、**PAI-EAS**、**GPU 加速 ECS 实例**、**容器服务 ACK**、**容器计算服务 ACS** 等，帮助你使用或部署 LLM 应用。作为云服务，它们具备高度的灵活性与可扩展性，是生产级 LLM 服务的首选方案，同时也能让你快速响应业务变化。

## 在 Linux 环境中部署（使用 vLLM）

先在小模型上演练部署流程总是个好主意。常见做法是使用 **Linux 环境**，因为你在云端也会依赖它。部署框架很多，但最流行的当属 **vLLM**——一个专为 LLM 打造、速度飞快的开源推理引擎。

可以把 vLLM 想象成模型的“试运行赛道”，它能够让你：

* 只需少量代码就能提供模型服务
* 在压力下评估性能表现
* 借助 OpenAI 兼容端点模拟真实 API 行为

我们将使用 `Qwen2.5-1.5B-Instruct` 模型——体量足够小，可在单块 GPU 上运行，却依然有不错的效果。

### 环境准备

我们继续使用 PAI-DSW，但在此之前需要切换到一台**带有 GPU 的实例**。

> 如果你已经在使用 GPU 加速的 PAI-DSW 实例，可以直接继续。  
> 若还没有，请先参考：`00 Setting Up the Environment`

准备就绪后，从 [HuggingFace 模型库](https://huggingface.co/models) 下载模型：

In [ ]:
from huggingface_hub import snapshot_download

# Specify the repository ID of the model on Hugging Face Hub
repo_id = "Qwen/Qwen2.5-1.5B-Instruct"

# Specify the local directory where you want to save the model
local_dir = "./model/qwen2_5-1_5b-instruct"

# Download the model
snapshot_download(repo_id=repo_id, local_dir=local_dir)

在开始使用模型之前，需要先安装 vLLM。

```bash
pip install vllm
```

然后在 vLLM 上启动模型服务。

```bash
vllm serve "./model/qwen2_5-1_5b-instruct" --load-format "safetensors" --port 8000
```

## 测试 vLLM 部署

vLLM 会启动一个兼容 OpenAI API 的本地服务，这意味着你可以用与 OpenAI 相同的格式，与自己托管的模型（例如 `Qwen2.5-1.5B-Instruct`）交互。你可以通过 `curl` 发送 HTTP 请求，来验证模型服务是否正常工作。

In [ ]:
%%bash
curl -s -X POST http://localhost:8000/v1/chat/completions \
    -H "Content-Type: application/json" \
    -d '{
        "model": "./model/qwen2_5-1_5b-instruct",
        "messages": [
            {"role": "system", "content": "You are TaskFriend, a helpful AI assistant that helps users manage daily tasks, prioritize work, and optimize time."},
            {"role": "user", "content": "I think I am burning out, can you help me?"}
        ]
    }'

如果收到成功响应，就说明本地的 vLLM 服务已经启动并运行良好。

此外，该服务还兼容 `/v1/models` 端点，可以列出当前部署的模型。更多信息请参考 vLLM 文档中的 [OpenAI-Compatible Server](https://docs.vllm.ai/en/latest/serving/openai_compatible_server.html)。

In [ ]:
%%bash
curl -X GET http://localhost:8000/v1/models

## 评估服务性能

接下来我们使用一个简单的 HTTP 压测工具 `wrk`，模拟负载请求并生成性能报告，从而评估已部署模型服务的表现。以下示例演示了如何针对 `POST /v1/chat/completions` 端点进行压测，并观察关键性能指标。

### 第一步：安装 `wrk`

回到终端，安装 `wrk`：

```bash
sudo apt update
sudo apt install wrk
```

### 第二步：准备 POST 请求体

准备好 POST 请求的 body 数据。我们已经在 `./resources/post.lua` 中帮你写好了，内容如下：

```json
wrk.method = "POST"
wrk.headers["Content-Type"] = "application/json"
wrk.body = [[
    {
       "model": "./model/qwen2_5-1_5b-instruct",
       "messages": [
           {"role": "system", "content": "You are a helpful AI assistant."},
           {"role": "user", "content": "Say hi!"}
       ]
   }
]]
```

### 第三步：使用 `wrk` 进行压测

在终端执行 `wrk` 压测命令。下面示例进行了两组测试，分别设置不同的并发数（`-c`），每次持续 20 秒（`-d 10s`），以观察服务在不同压力下的表现。


In [ ]:
%%bash

echo "Test 1: concurrency = 1"
echo "----------------------------"
wrk -t1 -c1 -d10s -s ./resources/post.lua http://localhost:8000/v1/chat/completions
echo -e "\n\n"

echo "Test 2: concurrency = 20"
echo "----------------------------"
wrk -t1 -c20 -d10s -s ./resources/post.lua http://localhost:8000/v1/chat/completions


### 第四步：解读压测结果

你应该会看到类似下面的测试结果：

<div style="text-align: center;">
  <img src="images/LMP-C01_09-StressTest.png" style="max-width: 800px;" />
  <br>
  <small>本地 vLLM 服务的压测结果</small>
</div>

根据测试结果可以看到，当并发从 `1` 提升到 `20` 时：

* **平均延迟** 大约增加了 50%（从 539.36ms 到 794.34ms）
* **QPS（每秒查询数）** 大约提升了 12 倍（从 1.80 提升到 24.07）

在更高的负载下，可能会出现超时。不过要注意：超时也可能意味着 `wrk` 没有在限定时间内接收完整响应（例如模型输出过长，未能在设定时间内返回完毕）。

---

## 使用 PAI-EAS 部署模型

**弹性算法服务（PAI-EAS）** 是 **PAI（Platform for AI）** 的一部分，为开源或自研模型提供生产级在线推理平台。PAI-EAS 支持多项高级能力：

* 自动扩缩容
* 蓝绿发布
* 版本管理
* 资源监控
* 模型预热（消除冷启动）

这些能力让 PAI-EAS 特别适合需要**低延迟、高可用**的实时同步推理场景。

对于成本敏感型应用，你可以选择使用 PAI-EAS Spot。它利用可中断实例，在保持容错性的前提下以更低成本运行推理任务。更多信息参见：[PAI-EAS Spot 最佳实践](https://www.alibabacloud.com/help/en/pai/use-cases/pai-eas-spot-best-practices)。

> **部署指南：**  
> 快速上手请参考 [Quickly deploy LLMs in EAS](https://www.alibabacloud.com/help/en/pai/use-cases/deploy-llm-in-eas)。

---

## 使用 ECS 或容器服务部署模型

在部署 LLM 时，**弹性计算服务（ECS）** 与基于容器的平台（如 ACK 或 ACS）是最常见的选择之一，它们能让你完全掌控部署环境。此类方案特别适合以下需求的模型：
- 特定的操作系统配置
- 自定义软件依赖
- 精细的性能调优
- 长时间运行、吞吐量高的推理服务

ECS 能够提供持续 7x24 小时的服务，还可以集成：
- **弹性伸缩（Auto Scaling）** 实现自动扩缩容
- **服务器负载均衡（SLB）** 提升高可用性
- **安全组与专有网络（VPC）** 增强安全防护

阿里云还为熟悉 Kubernetes 和容器环境的用户提供了 **容器服务 Kubernetes 版（ACK）** 与 **容器计算服务（ACS）**。根据团队偏好，可选择半托管或全托管模式。

虽然这些服务提供了最大化的灵活性，但也意味着对技术能力和运维投入的要求更高，因此需要结合场景仔细评估适用性：

| 适配度  | 典型场景   |
|--------------|-------------|
| 高         | <ul><li>需要稳定、长期性能的大模型</li><li>希望完全掌控资源与成本的企业</li><li>具备自定义优化的复杂推理流程</li></ul>  |
| 低          | <ul><li>快速原型开发或资源有限的团队</li><li>需要快速上线且希望极简运维的项目</li></ul>  |

> **部署参考：**
> * 弹性计算服务  
>        [在异构机密计算实例上构建支持安全度量的 DeepSeek 推理环境](https://www.alibabacloud.com/help/en/egs/use-cases/build-a-secure-deepseek-inference-environment-on-gn8v-tee-related-instances)
> * 容器类平台 
>     * 容器服务 Kubernetes 版：  
>        [在 ACK 中快速部署模型](https://www.alibabacloud.com/help/en/ack/cloud-native-ai-suite/use-cases/quickly-deploy-a-model-in-ack)  
>        [在 ACK 中使用 vLLM 部署 Qwen 模型推理服务](https://www.alibabacloud.com/help/en/ack/cloud-native-ai-suite/use-cases/deploy-qwen-model-inference-service-by-using-vllm-based-on-ack)
>    [在 ACK 多节点集群中部署 DeepSeek 全量能力的最佳实践](https://www.alibabacloud.com/help/en/ack/cloud-native-ai-suite/use-cases/distributed-deployment-of-full-capability-deepseek-inference-on-ack-multi-node-clusters)
>     * 容器计算服务：  
>        [在 GPU 加速的 ACS 集群上部署 Qwen3-32B](https://www.alibabacloud.com/help/en/cs/user-guide/build-qwen3-32b-model-inference-service-using-acs-gpu-computing-power)
>        [在 ACS 中基于 Dify 构建 AI 应用](https://www.alibabacloud.com/help/en/cs/user-guide/build-customized-ai-q-a-assistant-based-on-dify-in-acs)

---

## 我应该选择哪些服务？

在阿里云上使用或部署模型时，挑选合适的服务需要综合考量**业务需求、模型特性、团队技能、运维复杂度**以及**成本**。

下表提供了选择阿里云服务时的简明参考：

| 需求 | Model Studio | PAI-EAS | ECS | ACK/ACS |
|-------------|--------------|---------|-----|---------|
| 最佳适用场景 | 直接调用 Qwen 与 Wan 模型的 API | 标准 NLP/视觉模型部署（性能与易用性均衡） | 自定义环境、复杂依赖、完全掌控 | 高级 DevOps、微服务与可扩展 AI 平台 |
| 模型兼容性 | <ul><li>Qwen</li><li>Wan</li></ul> | 全部 | 全部 | 全部 |
| 运维复杂度 | ⭐<br>零运维（全托管、可视化界面） | ⭐⭐<br>低运维（托管推理，配置简单） | ⭐⭐⭐<br>高运维（需自管 OS、驱动与运行时） | ⭐⭐⭐⭐<br>高运维（集群管理、CI/CD、网络） |
| 团队技能要求 | 非技术用户、产品团队 | 算法工程师、ML 从业者 | DevOps、系统管理员、ML 工程师 | 经验丰富的 DevOps、平台工程师 |
| 成本模式 | 按调用计费 | 按量计费 | 按量或包年包月 | 集群 + 节点 + 管理成本 |
| 应用场景 | 面向大众的 AI 应用、快速原型 | 生产级 API、实时推理 | 模型研发、特殊硬件需求 | 多模型平台、CI/CD 流水线、混合式 AI 系统 |

> **免责声明：** 本表仅供参考。随着服务或需求的变化，结论可能会发生调整。

你也可以借助下方决策树，帮助自己为 LLM 应用选择合适的阿里云服务：

```mermaid
graph TD
    A["起点：我要部署 LLM"] --> B{"核心诉求是什么？"}

    B --> C["快速原型（如聊天机器人、助手）"]
    B --> E["标准模型部署（图像、文本、NLP）"]
    B --> F["需要自定义环境或复杂依赖"]

    C --> G["Model Studio<br><br>✓ 适合 Qwen 模型<br>✓ 适合 Wan 模型<br>✓ 无需运维<br>✓ 快速上线"]
    E --> I["选择 PAI-EAS<br><br>✓ 性能与易用性平衡<br>✓ 托管推理<br>✓ 支持常见框架"]

    F --> J{"团队是否具备较强能力？"}
    J -->|是| K["ECS、ACK 或 ACS<br><br>✓ 完全掌控<br>✓ 自定义镜像与扩缩容<br>✓ 适合进阶团队"]
    J -->|否| L["考虑使用 PAI-EAS<br>或 Model Studio<br>以减轻管理难度"]

    style G fill:#D6EAF8,stroke:#1F618D,color:#154360
    style I fill:#FEF9E7,stroke:#F4D03F,color:#9A7D0A
    style K fill:#E8DAEF,stroke:#884EA0,color:#4A235A
```

> **免责声明：** 上述决策树仅供参考。随着服务或需求的变化，建议也可能随之调整。

# 接下来做什么？

---

## 小测验

<details>
<summary style="cursor: pointer; padding: 12px; border: 1px solid #dee2e6; border-radius: 6px;">
<b>1. 什么时候应该选择在云端自托管，而不是使用像 Model Studio 这样的托管 API？</b>  

<ul>
    <li>A) 当你想以最快速度上线且无需运维时</li>
    <li>B) 当你需要完全掌控数据、模型与扩缩容时</li>
    <li>C) 当你只是在本地测试且不要求高可用时</li>
    <li>D) 当你使用的是 vLLM 不支持的模型时</li>
</ul>

**查看答案 →**
</summary>

<div style="margin-top: 10px; padding: 15px; border: 1px solid #dee2e6; border-radius: 0 0 6px 6px;">

✅ **正确答案：** B) 当你需要完全掌控数据、模型与扩缩容时  
📝 **解析**：  
* 自托管可以让你完全掌控基础设施、数据隐私与性能调优。
* 当无法依赖有速率限制或“黑盒”托管服务时，自托管会更合适。
* 托管 API（如 Model Studio）更适合快速原型。

</div>
</details>


## 关键要点

* **并非总需要部署**
    * **使用托管服务（例如 Model Studio）时无需部署**——模型已经托管并可自动扩缩容。
    * **这非常适合原型开发、学习或低到中等流量的应用。**
    * **优势包括零运维、按量付费与自动扩缩容。**
    * **权衡点**：存在 QPM/TPM 速率限制，对基础设施掌控力较弱。

<br>

* **何时选择云端自托管**
    * **当你需要完全掌控、保障数据隐私或做自定义优化时，请选择自托管。**
    * **典型场景：**
      - 微调或自研模型
      - 高吞吐量的生产系统
      - 对合规性要求严格的环境
    * **你需要负责扩缩容、监控与可靠性。**
    * **更适合具备 DevOps 或 MLOps 能力的团队。**

<br>

* **vLLM**
    * **vLLM 是一款面向 LLM 的开源高效推理引擎。**
    * **核心特性：**
      - **PagedAttention**：减少内存浪费、提升吞吐量
      - **循环批处理（Continuous batching）**：高效处理多请求
      - **兼容 OpenAI API**：易于与现有工具集成
    * **既适用于本地测试，也适用于生产部署。**

<br>

* **部署选项**
    * **Model Studio（托管）**：
      - 无需部署
      - 直接通过 API 访问
      - 特别适合阿里云通义系列模型
    * **本地 vLLM**：
      - 在本地机器运行
      - 上手快，适合测试
      - 不具备扩展能力
    * **云端部署（PAI-EAS、ECS、ACK）**：
      - 支持弹性、高并发推理
      - 完全掌控镜像、扩缩容与网络
      - 适用于企业级应用

<br>

* **如何择优**
    * **先用托管 API 验证产品，避免过早陷入基础设施复杂度。**
    * **当有性能、掌控或隐私需求时，再迁移到自托管。**
    * **使用 vLLM 获取高吞吐、低延迟的推理能力，同时保留灵活性。**
    * **先用较小的模型（如 1.5B 参数）做验证，再逐步放大。**
    * **在生产环境持续监测 GPU 利用率、延迟与请求错误。**
